# Two-Stage Stochastic Generation Expansion Planning

- Universidad Pontificia Comillas
- Master's Degree in the Electric Power Industry (MEPI)
- Diego Alejandro Tejada Arango

In [1]:
# Import packages
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import highspy

In [2]:
# General parameters
## folders names
input_folder  = 'inputs'
output_folder = 'outputs'

## solver definitions
SolverName = 'appsi_highs' #gurobi

In [3]:
## We define an abstract model
mGEP = pyo.AbstractModel()

In [4]:
# Sets and parameters of the abstract model

## sets
mGEP.p  = pyo.Set() #time periods (e.g., hours)
mGEP.sc = pyo.Set() #uncertainty scenarios
mGEP.g  = pyo.Set() #generation technologies

## parameters
mGEP.pScProb = pyo.Param(mGEP.sc) #scenario probability [p.u.]
mGEP.pDemand = pyo.Param(mGEP.p ) #demand per time period [MW]

mGEP.pVarCost = pyo.Param(mGEP.g) #variable   cost of generation units [kEUR/MWh]
mGEP.pInvCost = pyo.Param(mGEP.g) #investment cost of generation units [kEUR/MW/year]
mGEP.pUnitCap = pyo.Param(mGEP.g) #capacity        of generation units [MW]
mGEP.pIsRenew = pyo.Param(mGEP.g) #renewable units boolean indicator   [0,1]

mGEP.pWeight  = pyo.Param(within=pyo.NonNegativeReals) #weight of representative period [days]
mGEP.pENSCost = pyo.Param(within=pyo.NonNegativeReals) #energy not supplied cost    [kEUR/MWh]

mGEP.pAviProf = pyo.Param(mGEP.sc,mGEP.g,mGEP.p,default=1,mutable=True)   #availability profile [p.u.]

In [5]:
# Variables of the abstract model
mGEP.vInvesCost   = pyo.Var(                      domain=pyo.NonNegativeReals, doc='Total investment Cost                [kEUR]')
mGEP.vOperaCost   = pyo.Var(                      domain=pyo.NonNegativeReals, doc='Total operating  Cost                [kEUR]')
mGEP.vProduct     = pyo.Var(mGEP.sc,mGEP.g,mGEP.p,domain=pyo.NonNegativeReals, doc='generation production per scenario   [MW]  ')
mGEP.vInstalUnits = pyo.Var(        mGEP.g,       domain=pyo.Integers        , doc='number of installed generation units [N]   ')
mGEP.vENS         = pyo.Var(mGEP.sc,       mGEP.p,domain=pyo.NonNegativeReals, doc='energy not supplied   per scenario   [MW]  ')

In [6]:
# Objective function of the abstract model
def eTotalCost(model):
  return   model.vInvesCost + model.vOperaCost
mGEP.eTotalCost = pyo.Objective(rule=eTotalCost)

In [7]:
# Constraints

## Total investment Cost [kEUR]
def eInvesCost(model):
  return model.vInvesCost == sum(model.pInvCost[g]*model.pUnitCap[g]*model.vInstalUnits[g] for g in model.g)
mGEP.eInvesCost = pyo.Constraint(rule=eInvesCost)

## Total operating  Cost [kEUR]
def eOperaCost(model):
  return model.vOperaCost == (model.pWeight*(sum(model.pScProb[sc]*model.pVarCost[g]*model.vProduct[sc,g,p] for sc,g,p in model.sc*model.g*model.p) +
                                             sum(model.pScProb[sc]*model.pENSCost   *model.vENS    [sc,  p] for sc,  p in model.sc*        model.p)))
mGEP.eOperaCost = pyo.Constraint(rule=eOperaCost)

## Power balance constraint [MW]
def eBalance(model,sc,p):
  return sum(model.vProduct[sc,g,p] for g in model.g) + model.vENS[sc,p] == model.pDemand[p]
mGEP.eBalance = pyo.Constraint(mGEP.sc,mGEP.p,rule=eBalance)

## Max generation constraint
def eMaxProd(model,sc,g,p):
  return model.vProduct[sc,g,p] <= model.pAviProf[sc,g,p]*model.pUnitCap[g]*model.vInstalUnits[g]
mGEP.eMaxProd = pyo.Constraint(mGEP.sc,mGEP.g,mGEP.p,rule=eMaxProd)

## Max ENS constraint
def eENSProd(model,sc,p):
  return model.vENS[sc,p] <= model.pDemand[p]
mGEP.eENSProd = pyo.Constraint(mGEP.sc,mGEP.p,rule=eENSProd)

In [8]:
# We define the optimization solver. You can also use cplex, gurobi, etc

opt = SolverFactory(SolverName)

## We define the options of the solver (this depends on the solver you are using)
opt.options['mip_rel_gap'] = 0 # HiGHS option for relative gap

In [9]:
# We open a DataPortal to load the data
data = pyo.DataPortal() 

## We read all the data from different files
### Scalars
data.load(filename='./'+input_folder+'/scalars.dat')   

### Sets
data.load(filename='./'+input_folder+'/oGEP_Data_Demand.csv'    ,format='set', set='p' )
data.load(filename='./'+input_folder+'/oGEP_Data_Generation.csv',format='set', set='g' )
data.load(filename='./'+input_folder+'/oGEP_Data_Scenario.csv'  ,format='set', set='sc')

### Parameters
data.load(filename='./'+input_folder+'/oGEP_Data_Demand.csv'    ,index=          'p' , param= 'pDemand' )
data.load(filename='./'+input_folder+'/oGEP_Data_Generation.csv',index=      'g'     , param=['pVarCost','pInvCost','pUnitCap','pIsRenew'])
data.load(filename='./'+input_folder+'/oGEP_Data_Scenario.csv'  ,index= 'sc'         , param= 'pScProb' )
data.load(filename='./'+input_folder+'/oGEP_Data_GenAviProf.csv',index=['sc','g','p'], param= 'pAviProf')

In [10]:
# We create an instance  
instance = mGEP.create_instance(data)

## We can display all the info of the instance
instance.pprint()

## write the optimization problem
instance.write('mGEP.lp', io_options={'symbolic_solver_labels': True})

9 Set Declarations
    eBalance_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :   sc*p :   72 : {('sc1', 'h01'), ('sc1', 'h02'), ('sc1', 'h03'), ('sc1', 'h04'), ('sc1', 'h05'), ('sc1', 'h06'), ('sc1', 'h07'), ('sc1', 'h08'), ('sc1', 'h09'), ('sc1', 'h10'), ('sc1', 'h11'), ('sc1', 'h12'), ('sc1', 'h13'), ('sc1', 'h14'), ('sc1', 'h15'), ('sc1', 'h16'), ('sc1', 'h17'), ('sc1', 'h18'), ('sc1', 'h19'), ('sc1', 'h20'), ('sc1', 'h21'), ('sc1', 'h22'), ('sc1', 'h23'), ('sc1', 'h24'), ('sc2', 'h01'), ('sc2', 'h02'), ('sc2', 'h03'), ('sc2', 'h04'), ('sc2', 'h05'), ('sc2', 'h06'), ('sc2', 'h07'), ('sc2', 'h08'), ('sc2', 'h09'), ('sc2', 'h10'), ('sc2', 'h11'), ('sc2', 'h12'), ('sc2', 'h13'), ('sc2', 'h14'), ('sc2', 'h15'), ('sc2', 'h16'), ('sc2', 'h17'), ('sc2', 'h18'), ('sc2', 'h19'), ('sc2', 'h20'), ('sc2', 'h21'), ('sc2', 'h22'), ('sc2', 'h23'), ('sc2', 'h24'), ('sc3', 'h01'), ('sc3', 'h02'), ('sc3', 'h03'), ('sc3', 'h04'), ('sc3', 

('mGEP.lp', 2036126861840)

In [11]:
# We solve the optimization problem
results = opt.solve(instance,symbolic_solver_labels=True,tee=True)

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [7e-02, 2e+04]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [8e+02, 1e+03]
Presolving model
324 rows, 328 cols, 828 nonzeros  0s
324 rows, 328 cols, 828 nonzeros  0s

Solving MIP model with:
   324 rows
   328 cols (0 binary, 4 integer, 0 implied int., 324 continuous)
   828 nonzeros
MIP-Timing:      0.0023 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Ti

In [12]:
# Print results

## Print the number of variables and constraints
print("Number of variables: "+str(instance.nvariables()))
print("Number of constraints: "+str(instance.nconstraints()))

## Check if the problem is optimal
if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
  ### objective function value
  print("total cost: "+str(instance.vInvesCost.value+instance.vOperaCost.value))
  ### We write some of the results in a csv file
  f = open('./'+output_folder+'/oGEP_Invest_Result.csv', 'w')
  f.write("g,vInstalUnits,pInstalCap"+"\n")
  for g in instance.g.data():
    f.write(str(g)+","+str(instance.vInstalUnits[g].value)+","+str(instance.pUnitCap[g]*instance.vInstalUnits[g].value)+"\n")
  f.close()
else:
  ### Print a message indicating that the problem is not optimal
  print("The problem is not optimal.")
  ### Print the solver status
  print("Solver Status: "+str(results.solver.status))

Number of variables: 366
Number of constraints: 434
total cost: 269238.43825
